Tut 1.4: Reaction $$2A\ \rightarrow B$$.  Pure A fed, I.e. expansion factor = -0.5 

Inlet pressure = 20 atm, outlet = 5 atm, W = 1 kg

Conversion of A = 30%


Use PBR information to determine rate constant and Ergun constant 

Since $$F_{Ao} $$  and $$Q_{o}$$  not given create one grouped constant to be used in rate expression:  $$k = k(\frac{F_{Ao}}{Q_{O}})^{2}$$.  It should also work if you just choose a value of 1 for both variables

In [1]:
import numpy as np
import scipy as sp
import scipy.optimize as spop
import scipy.integrate as sci

Define parameters:

In [2]:
Po = 20; #atm
ex_factor = -0.5;  #2A - B
Pout = 5; #atm
xfin = 0.3; #atm
W = 1; # kg
Wspan = np.linspace(0,1,20)
inlet = [0,20]; #inlet conversion and pressure
guess = [1, 4.4] #First guess values for k and Ergun constant (K) in atm/kg. Use DelP/DelW = -K(1+eps.xfin) to get value for K

Get ballpark guess value for Ergun constant:

In [3]:
delP = Pout - inlet[1]
delW = 1
K = -(delP)/delW/(1+ex_factor*xfin)/(Po/Pout) #If the outlet slope defines slope, you can also use x = 0 and Po/Pout = 1 for guess
k = 1;
guess = [k,K]

Define a function that takes in values for k and K, use them to integrate over W span (1 kg) and compare x model to the actual x at 1 kg as well as the Pout model versus the actual Pout at 1 kg.  Need two equations to solve for two unknowns.  Will define errors between actual and model outcomes for both x and P

In [10]:
def eqs(constants):
    K1, K2 = constants
    
    def DEs(var,W):
        x, P = var
        dxdW = K1*(1-x)**2/(1+ex_factor*x)**2*(P/Po)**2
        dPdW = -K2*(1+ex_factor*x)*(Po/P)
        return [dxdW, dPdW]
    xP_model = sci.odeint(DEs, inlet, Wspan)
    outlet = xP_model[-1]
    Error1 = (outlet[0] - xfin)**2*100
    Error2 = (outlet[1] - Pout)**2
    return [Error1, Error2]

In [11]:
Ks = spop.fsolve(eqs,guess,full_output =1)
print(Ks)  # use full-output to make sure that solution was reached


(array([  0.69492223,  10.43425483]), {'fvec': array([  3.91889581e-14,   1.65826055e-14]), 'r': array([ -4.01825955e-06,  -1.67377943e-06,  -1.94877134e-06]), 'qtf': array([ -1.61646044e-13,   5.27118760e-14]), 'nfev': 62, 'fjac': array([[-0.77147123, -0.63626421],
       [ 0.63626421, -0.77147123]])}, 1, 'The solution converged.')


In [16]:
constants = Ks[0]
print('Values of the constants are: k = ' + str(constants[0]) + ' and K  = ' + str(constants[1]) + ' atm/kg')

Values of the constants are: k = 0.694922229713 and K  = 10.4342548318 atm/kg
